In [1]:
import os
# 【硬性要求】加载环境变量，确保能读取到 OPENAI_API_KEY
from dotenv import load_dotenv
load_dotenv(override=True)

# 旧版本：from langchain_openai.chat_models import ChatOpenAI
# 现在更符合现代工程标准的写法是直接从 langchain_openai 导入，
# 因为官方统一了模块层级，使导入路径更加简洁清晰。
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ==========================================
# 1. 初始化模型
# ==========================================
# 明确指定模型版本，在科研复现中这非常重要，避免默认模型随时间暗中升级导致结果突变
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0) 

# ==========================================
# 2. 构建带有记忆插槽的 Prompt Template
# ==========================================
prompt = ChatPromptTemplate.from_messages(
    [
        # System 消息：设定 Agent 的人设和全局规则
        ("system", "You're an assistant who's good at {ability}. Respond in 20 words or fewer."),
        
        # 【核心魔法】：消息占位符
        # 它会在运行时被替换为一个包含多条 HumanMessage 和 AIMessage 的列表。
        # variable_name="history" 意味着调用链时，你需要传入一个键为 "history" 的变量。
        MessagesPlaceholder(variable_name="history"),
        
        # Human 消息：当前用户的最新输入
        ("human", "{input}"),
    ]
)

# ==========================================
# 3. 组装基础链
# ==========================================
base_chain = prompt | model

# ==========================================
# 4. 模拟对话历史并执行调用
# ==========================================
if __name__ == "__main__":
    # 【科研级保姆注释】
    # 这里的 history 是硬编码的“短期记忆”。
    # 在现代 LangChain 生产环境中，我们通常不手动维护这个 List，
    # 而是使用 `RunnableWithMessageHistory` 类结合 Redis/SQLite 来自动管理历史状态。
    # 但为了展示底层逻辑，我们保留老师的手动传入方式：
    chat_history = [
        ("human", "What's a right-angled triangle?"),
        ("ai", "A right-angled triangle has one angle of 90 degrees, with the other two angles summing to 90 degrees.")
    ]
    
    # 第一次提问：依赖上下文中的 "right-angled triangle"
    new_question = "What are the other types?"
    
    print(f"当前问题: {new_question}")
    print("正在结合历史记忆进行推理...\n")
    
    # 触发调用：传入人设变量 (ability)、新问题 (input) 以及历史记录 (history)
    response = base_chain.invoke(
        {
            "ability": "math", 
            "input": new_question, 
            "history": chat_history
        }
    )
    
    print("=== AI 回复 ===")
    print(response.content)

当前问题: What are the other types?
正在结合历史记忆进行推理...

=== AI 回复 ===
Other types of triangles include equilateral (all sides equal), isosceles (two sides equal), and scalene (no sides equal).
